In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
#Advanced Fork-Join Example 13
#Complex fork-join with 3 parallel branches
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
queue3 = Queue(model,'Queue3',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
# Class 1
queue1.set_service(jobclass1, Exp(1.5))
queue2.set_service(jobclass1, Exp(1.1))
queue3.set_service(jobclass1, Exp(2.5))
delay.set_service(jobclass1, Exp(0.5))

# Class 2
queue1.set_service(jobclass2, Exp(2.8))
queue2.set_service(jobclass2, Exp(3.0))
queue3.set_service(jobclass2, Exp(1.0))
delay.set_service(jobclass2, Exp(0.8))

In [5]:
# Set routing matrix with serial routing queue2->queue3
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # Serial: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue2, queue3, 1.0)  # Serial: queue2 -> queue3
P.set(jobclass2, jobclass2, queue3, join, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with MVA only (like MATLAB)
solver = SolverMVA(model)
avgTable = solver.get_avg_table()

MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.149325s.
  Station JobClass    QLen    Util    RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.4649  1.4649   2.0000  2.0000  0.7325  0.7325
1  Delay1   class2  0.7901  0.7901   1.2500  1.2500  0.6321  0.6321
2  Queue1   class1  1.6386  0.4883   2.2371  1.1185  0.7325  0.7325
3  Queue1   class2  0.7575  0.2257   1.1984  0.5992  0.6321  0.6321
4  Queue2   class1  4.7629  0.6659   6.5026  3.2513  0.7325  0.7325
5  Queue2   class2  1.5071  0.2107   2.3843  1.1922  0.6321  0.6321
6  Queue3   class1  3.5643  0.2930   4.8662  2.4331  0.7325  0.7325
7  Queue3   class2  7.6896  0.6321  12.1655  6.0827  0.6321  0.6321
8    Join   class1  7.2282  0.0000   4.9342  4.9342  1.4649  0.7325
9    Join   class2  8.5556  0.0000   6.7677  6.7677  1.2642  0.6321
